# Make Web Site

In [52]:
import sys, os, io, datetime
import json
import torch
import random
import requests
import zipfile
import pandas as pd
from slugify import slugify
from tqdm.notebook import tqdm

In [3]:
import cdli

In [2]:
pd.set_option("display.max_columns", None)

In [27]:
supported_langs = set(["akk", "sux"])

In [44]:
wwwroot = os.path.abspath("../dist")
os.makedirs(wwwroot)
wwwroot

'/Users/fak/Dropbox/Projects/CuneiformTranslators/dist'

## Download the Catalog

In [4]:
cat = cdli.get_catalog()

/Users/fak/Dropbox/Projects/CuneiformTranslators/tools/cdli.py:95: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,13,14,16,17,19,22,23,24,25,26,27,28,29,31,32,33,37,39,42,45,46,49,52,53,54,55,56,57,58,59,60,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  cat = pd.read_csv(io.StringIO(cat_csv))


In [5]:
cat

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
0,NaN,NaN,NaN,NaN,21198/zz001q0dtm,"Englund, Robert K.",NaN,CDLI,"31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2020-03-14,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 065",NaN,NaN,NaN,NaN,"W 06435,a",NaN,auf Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,31,1,0,1,NaN,undetermined,150ppi 20160630,clay,VAT 01533,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 065",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,18,no translation,61,NaN
1,NaN,NaN,NaN,NaN,21198/zz001q0dv4,"Englund, Robert K.",NaN,CDLI,30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 066",NaN,NaN,NaN,NaN,"W 06435,b",NaN,auf der Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,30,2,0,2,NaN,undetermined,150ppi 20160630,clay,VAT 15263,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 066",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,13,no translation,48,NaN
2,NaN,NaN,NaN,NaN,21198/zz001q0dwn,"Englund, Robert K.",NaN,"Englund, Robert K. & Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",NaN,NaN,12/4/2001,NaN,NaN,2020-01-26,NaN,20011204 protocuneiform_catalogue,"ATU 3, pl. 081, W 9123,d",NaN,NaN,NaN,NaN,"W 09123,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,42,3,0,3,NaN,undetermined,150ppi 20160630,clay,VAT 15253,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"ATU 3, pl. 081, W 9123,d",Uruk (mod. Warka),NaN,1993,"ATU 1, 539",NaN,NaN,NaN,NaN,Archaic Vocabulary (witness),Text category: 15-09; Foreign ID: LVO 9,NaN,NaN,19,no translation,53,NaN
3,NaN,NaN,NaN,NaN,21198/zz001q0dx5,"Englund, Robert K.",NaN,CDLI,26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 051",NaN,NaN,NaN,NaN,"W 09169,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,26,4,0,4,NaN,undetermined,150ppi 20160630,clay,VAT 15168,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"CDLI Lexical 000002, ex. 051",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,23,no translation,23,NaN
4,NaN,NaN,NaN,NaN,21198/zz001q0dzp,"Englund, Robert K.",NaN,CDLI,"29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 172",NaN,NaN,NaN,NaN,"W 09206,k",

In [53]:
len(cat)

352771

In [6]:
cat.columns

Index(['accession_no', 'accounting_period', 'acquisition_history',
       'alternative_years', 'ark_number', 'atf_source', 'atf_up', 'author',
       'author_remarks', 'cdli_collation', 'cdli_comments', 'citation',
       'collection', 'composite_id', 'condition_description', 'date_entered',
       'date_of_origin', 'date_remarks', 'date_updated', 'dates_referenced',
       'db_source', 'designation', 'dumb', 'dumb2', 'electronic_publication',
       'elevation', 'excavation_no', 'external_id', 'findspot_remarks',
       'findspot_square', 'genre', 'google_earth_collection',
       'google_earth_provenience', 'height', 'id', 'id_text2', 'id_text',
       'join_information', 'language', 'lineart_up', 'material', 'museum_no',
       'object_preservation', 'object_type', 'period', 'period_remarks',
       'photo_up', 'primary_publication', 'provenience', 'provenience_remarks',
       'publication_date', 'publication_history', 'published_collation',
       'seal_id', 'seal_information', 's

In [7]:
genres = sorted(list(set([str(x).replace("?", "").strip() for x in cat["genre"].unique()])))
genres

['Administrative',
 'Administrative Record',
 'Astronomical',
 'Astronomical, Omen',
 'Historical',
 'Legal',
 'Letter',
 'Lexical',
 'Lexical; Literary',
 'Lexical; Literary; Mathematical',
 'Lexical; Mathematical',
 'Lexical; School',
 'Literary',
 'Literary; Administrative',
 'Literary; Letter',
 'Literary; Lexical',
 'Literary; Mathematical',
 'Mathematical',
 'Medical',
 'Omen',
 'Omens',
 'Other (see subgenre)',
 'Pottery (seal)',
 'Prayer/Incantation',
 'Private/Votive',
 'Ritual',
 'Royal/Monumental',
 'Royal/Monumental; Literary',
 'Royal/Votive',
 'School',
 'School; Literary',
 'Scientific',
 'Uncertain',
 'Votive',
 'administative',
 'administrative',
 'fake (modern)',
 'legal',
 'letter',
 'lexical',
 'literary',
 'nan',
 'school',
 'scientific',
 'uncertain']

https://cdli.ucla.edu/search/archival_view.php?ObjectID=P256681

In [8]:
cat[cat["id_text"]==256681]

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
106190,NaN,NaN,NaN,NaN,21198/zz001s1m1w,"de Ridder, Alba",NaN,"Stol, Marten",NaN,NaN,NaN,NaN,University of Pennsylvania Museum of Archaeolo...,NaN,NaN,2/24/2005,NaN,NaN,2021-06-12,NaN,20050224 fitzgerald_upenn,"AbB 11, 029",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Letter,NaN,NaN,?,106192,0,256681,NaN,Akkadian,NaN,clay,UM 29-16-076,NaN,tablet,Old Babylonian (ca. 1900-1600 BC),NaN,600ppi 20160630,"AbB 11, 029",Nippur (mod. Nuffar),NaN,1986,NaN,NaN,NaN,NaN,NaN,NaN,Letter; 7x8x1 line,NaN,NaN,?,no translation,?,NaN


## Get the Transliterations

In [9]:
transliterations = cdli.get_atf()

100%|██████████████████████████████████████████████████████████████████████████████████| 3541243/3541243 [00:14<00:00, 251605.89it/s]


[Publication('P000001', 'qpc', [TextArea('tablet', []), TextArea('obverse', []), TextArea('column 1', [TextLine("1'.", '1(N01) , [...]', {}), TextLine("2'.", '1(N01) , TIM ABGAL#', {}), TextLine("3'.", '1(N01) , KINGAL#', {})]), TextArea('column 2', [TextLine("1'.", '1(N01) , [...]', {}), TextLine("2'.", '1(N01) , GAL~a# UMUN2#', {}), TextLine("3'.", '1(N01) , GAL~a UMUN2 KU3~a', {})]), TextArea('column 3', [TextLine("1'.", '1(N01) , DUB~a SANGA~a#', {}), TextLine("2'.", '1(N01) , SUG5# SAG#', {}), TextLine("3'.", '1(N01) , UB SAG#', {})]), TextArea('reverse', [TextLine('1.', '[N] , [...]', {})])]),
 Publication('P000002', 'qpc', [TextArea('tablet', []), TextArea('obverse', []), TextArea('column 1', [TextLine("1'.", '[1(N01)] , [...]', {}), TextLine("2'.", '[1(N01)] , GAL~a# SZITA~a1', {}), TextLine("3'.", '[1(N01)] , ABGAL#', {})]), TextArea('column 2', [TextLine("1'.", '[1(N01)] , [...] NIM~a#?', {}), TextLine("2'.", '1(N01) , GAL~a SILA4~b', {}), TextLine("3'.", '1(N01) , GAL~a SZAB

## Get a list of publications that we have data for

In [51]:
output_pubs = []

for pub in tqdm(transliterations):
    if pub.language not in supported_langs:
        continue
    pid = pub.id
    if len(pid) < 3 or pid[0] != "P":
        continue
    if pid[-1] == "=":
        pid = pid[:-1]
    pid = int(pid[1:])
    meta = cat[cat["id_text"]==pid]
    if len(meta) != 1:
        continue
    pub.genre = str(meta["genre"].values[0])    
    pub.object_type = str(meta["object_type"].values[0])
    pub.material = str(meta["material"].values[0])
    pub.period = str(meta["period"].values[0])
    pub.provenience = str(meta["provenience"].values[0])
    pub.collection = str(meta["collection"].values[0])
#     language = str(meta["language"].values[0])
#     print(pub.genre)
    output_pubs.append(pub)

len(output_pubs)

  0%|          | 0/134536 [00:00<?, ?it/s]

121087

In [58]:
num_publications = len(cat)
num_transliterations = len(output_pubs)
num_translations = len([x for x in output_pubs if x.has_translations()])
print("    num_publications:", num_publications)
print("num_transliterations:", num_transliterations)
print("    num_translations:", num_translations)

    num_publications: 352771
num_transliterations: 121087
    num_translations: 5196


## Output the HTML

In [60]:
for pub in tqdm(output_pubs):
    pass

  0%|          | 0/121087 [00:00<?, ?it/s]